In [ ]:
source_videos_path = "/content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos"
destination_videos_path = "/content/drive/MyDrive/NLP_Project/cropped_outputs"
input_video_for_tarncribe = "input.mp4"
subtitle_language = 'tr'  # Turkish language code
maxlen = 79
num_classes = 7


softmax_of_text = '/content/drive/MyDrive/NLP_Project/softmax_output_of_text.txt'
softmax_of_video = '/content/drive/MyDrive/NLP_Project/softmax_output_of_video.txt'

video_path = destination_videos_path + "/" + input_video_for_tarncribe
output_folder = "/content/drive/MyDrive/NLP_Project/Photos_to_predict"


input_video_to_add_sentiment = "/content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos/output-translated.mp4"  # Get the uploaded video filename
output_video_with_audio = "Final_Video.mp4"


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%cd /content/drive/MyDrive/NLP_Project/testMarlin/MARLIN

/content/drive/MyDrive/NLP_Project/testMarlin/MARLIN


In [ ]:
!pip install ffmpeg
!pip install marlin_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=0b94e859e102ea71cc8d543f7f1c7c4acd21043f08fb1391b1bac7fcbddfbbd3
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 51.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)

In [ ]:
import glob
import logging.config
import os
import sys
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from typing import Tuple

import cv2
import ffmpeg
import yaml
from numpy import ndarray
from tqdm.auto import tqdm



from marlin_pytorch.util import crop_with_padding
from util.face_sdk.core.model_loader.face_detection.FaceDetModelLoader import FaceDetModelLoader
from util.face_sdk.core.model_handler.face_detection.FaceDetModelHandler import FaceDetModelHandler

In [ ]:
logging.config.fileConfig(os.path.join("util", "face_sdk", "config", "logging.conf"))
logger = logging.getLogger('api')

with open(os.path.join("util", "face_sdk", "config", "model_conf.yaml")) as f:
    model_conf = yaml.load(f, Loader=yaml.FullLoader)

# common setting for all model, need not modify.
model_path = os.path.join("util", "face_sdk", 'models')

# model setting, modified along with model
scene = 'non-mask'
model_category = 'face_detection'
model_name = model_conf[scene][model_category]

In [ ]:
logger.info('Start to load the face detection model...')
# load model
sys.path.append(os.path.join("util", "face_sdk"))
faceDetModelLoader = FaceDetModelLoader(model_path, model_category, model_name)
model, cfg = faceDetModelLoader.load_model()
faceDetModelHandler = FaceDetModelHandler(model, 'cuda:0', cfg)

INFO 2024-05-22 10:31:19 <ipython-input-6-87c6f92a1489>: 1] Start to load the face detection model...


/usr/local/lib/python3.10/dist-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'models.network_def.retinaface_def.RetinaFace' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torchvision.models._utils.IntermediateLayerGetter' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessin

In [ ]:
def crop_face(frame, margin=1, x=0, y=0) -> Tuple[ndarray, int, int, int]:
    assert frame.ndim == 3 and frame.shape[2] == 3, "frame should be 3-dim"
    dets = faceDetModelHandler.inference_on_image(frame)
    if len(dets) > 0:
        x1, y1, x2, y2, confidence = dets[0]
        # center
        x, y = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        margin = int(max(abs(x2 - x1), abs(y2 - y1)) / 2)
    # crop face
    face = crop_with_padding(frame, x - margin, x + margin, y - margin, y + margin, 0)
    face = cv2.resize(face, (224, 224))
    return face, margin, x, y


In [ ]:
def crop_face_video(video_path: str, save_path: str, fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=30) -> None:
    cap = cv2.VideoCapture(video_path)
    writer = cv2.VideoWriter(save_path, fourcc=fourcc, fps=fps, frameSize=(224, 224))
    x, y = 0, 0
    margin = 1

    while True:
        ret, frame = cap.read()
        if ret:
            face, margin, x, y = crop_face(frame, margin, x, y)
            writer.write(face)
        else:
            break

    cap.release()
    writer.release()


In [ ]:
def crop_face_img(img_path: str, save_path: str):
    frame = cv2.imread(img_path)
    face = crop_face(frame)[0]
    cv2.imwrite(save_path, face)


In [ ]:
def process_videos(video_path, output_path, ext="mp4", max_workers=8):
    if ext == "mp4":
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    elif ext == "avi":
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
    else:
        raise ValueError("ext should be mp4 or avi")

    Path(output_path).mkdir(parents=True, exist_ok=True)

    files = os.listdir(video_path)
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = []

        for f_name in tqdm(files):
            if f_name.endswith('.' + ext):
                source_path = os.path.join(video_path, f_name)
                target_path = os.path.join(output_path, f_name)
                fps = eval(ffmpeg.probe(source_path)["streams"][0]["avg_frame_rate"])
                futures.append(executor.submit(crop_face_video, source_path, target_path, fourcc,
                    fps))

        for future in tqdm(futures):
            future.result()


In [ ]:


def process_images(image_path: str, output_path: str, max_workers: int = 8):
    Path(output_path).mkdir(parents=True, exist_ok=True)
    files = glob.glob(f"{image_path}/*/*/*.jpg")
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = []

        for file in tqdm(files):
            save_path = file.replace(image_path, output_path)
            Path("/".join(save_path.split("/")[:-1])).mkdir(parents=True, exist_ok=True)
            futures.append(executor.submit(crop_face_img, file, save_path))

        for future in tqdm(futures):
            future.result()


In [ ]:
process_videos(source_videos_path, destination_videos_path, ext="mp4")

In [ ]:
%cd /content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos

/content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos


In [ ]:
!pip3 install faster-whisper ffmpeg-python

In [ ]:
!pip3 freeze > /content/requirements.txt

In [ ]:
import time
import math
import ffmpeg

from faster_whisper import WhisperModel

model = WhisperModel("large")



In [ ]:
input_video = input_video_for_tarncribe
input_video_name = input_video.replace(".mp4", "")

In [ ]:
def extract_audio(input_video, input_video_name):
    extracted_audio = f"audio-{input_video_name}.wav"
    stream = ffmpeg.input(input_video)
    stream = ffmpeg.output(stream, extracted_audio)
    ffmpeg.run(stream, overwrite_output=True)
    return extracted_audio

In [ ]:
extracted_audio = extract_audio(input_video, input_video_name)

In [ ]:
def transcribe(audio):
    segments, info = model.transcribe(audio)
    language = info[0]
    print("Transcription language", info[0])
    segments = list(segments)
    for segment in segments:
        # print(segment)
        print("[%.2fs -> %.2fs] %s" %
              (segment.start, segment.end, segment.text))
    return language, segments

In [ ]:
language, segments = transcribe(audio=extracted_audio)

Transcription language en
[0.00s -> 2.00s]  Hey, this is Ken from AES.
[2.00s -> 6.00s]  Sometimes we're asked about the difference between what we do and a textbook.
[6.00s -> 8.00s]  Let's talk about textbooks first.
[8.00s -> 13.00s]  The best thing about a textbook is you can buy it once and you can keep using it for three or four years.


In [ ]:
def format_time(seconds):

    hours = math.floor(seconds / 3600)
    seconds %= 3600
    minutes = math.floor(seconds / 60)
    seconds %= 60
    milliseconds = round((seconds - math.floor(seconds)) * 1000)
    seconds = math.floor(seconds)
    formatted_time = f"{hours:02d}:{minutes:02d}:{seconds:01d},{milliseconds:03d}"

    return formatted_time

In [ ]:
def generate_subtitle_file(language, segments):

    subtitle_file = f"sub.srt"
    text = ""
    for index, segment in enumerate(segments):
        segment_start = format_time(segment.start)
        segment_end = format_time(segment.end)
        text += f"{str(index+1)} \n"
        text += f"{segment_start} --> {segment_end} \n"
        text += f"{segment.text} \n"
        text += "\n"

    f = open(subtitle_file, "w")
    f.write(text)
    f.close()

    return subtitle_file

In [ ]:
def add_subtitle_to_video(soft_subtitle, subtitle_file,  subtitle_language, input_video, input_video_name):

    video_input_stream = ffmpeg.input(input_video)
    subtitle_input_stream = ffmpeg.input(subtitle_file)
    output_video = f"output-translated.mp4"
    subtitle_track_title = subtitle_file.replace(".srt", "")

    if soft_subtitle:
        stream = ffmpeg.output(
            video_input_stream, subtitle_input_stream, output_video, **{"c": "copy", "c:s": "mov_text"},
            **{"metadata:s:s:0": f"language={subtitle_language}",
            "metadata:s:s:0": f"title={subtitle_track_title}"}
        )
        ffmpeg.run(stream, overwrite_output=True)
    else:
        stream = ffmpeg.output(video_input_stream, output_video,
                                vf=f"subtitles={subtitle_file}")
        ffmpeg.run(stream, overwrite_output=True)

In [ ]:
# Install required libraries
!pip install googletrans==3.1.0a0


In [ ]:
segments, info = model.transcribe(extracted_audio)


In [ ]:
text = []
sentences_num = 0
for segment in segments:
  text.append(segment.text)
  sentences_num += 1


In [ ]:
text

[' Hey, this is Ken from AES.',
 " Sometimes we're asked about the difference between what we do and a textbook.",
 " Let's talk about textbooks first.",
 ' The best thing about a textbook is you can buy it once and you can keep using it for three or four years.']

In [ ]:
%cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:
# Open the file for writing
with open("sentences.txt", "w") as text_file:
  # Loop through each element in the array
  for line in text:
    print(line)
    # Write the line to the file with a newline character at the end
    text_file.write(line + "\n")

 Hey, this is Ken from AES.
 Sometimes we're asked about the difference between what we do and a textbook.
 Let's talk about textbooks first.
 The best thing about a textbook is you can buy it once and you can keep using it for three or four years.


In [ ]:
from googletrans import Translator




# Initialize translator
translator = Translator()
# Define translation language (replace 'fr' with your desired language code)
target_language = 'tr'

# Translate the text
translated = translator.translate(text, dest=target_language)
translated[0].text


"Merhaba, bu AES'ten Ken."

In [ ]:
translated_text = [element.text for element  in translated]
translated_text

["Merhaba, bu AES'ten Ken.",
 'Bazen bize yaptıklarımız ile bir ders kitabı arasındaki fark sorulur.',
 'Önce ders kitaplarından bahsedelim.',
 'Bir ders kitabının en iyi yanı, onu bir kere satın alıp üç ya da dört yıl boyunca kullanmaya devam edebilmenizdir.']

In [ ]:



# Print the translated text
print(f"Original Text: {text}")
print(f"Translated Text ({target_language}): {translated_text}")

Original Text: [' Hey, this is Ken from AES.', " Sometimes we're asked about the difference between what we do and a textbook.", " Let's talk about textbooks first.", ' The best thing about a textbook is you can buy it once and you can keep using it for three or four years.']
Translated Text (tr): ["Merhaba, bu AES'ten Ken.", 'Bazen bize yaptıklarımız ile bir ders kitabı arasındaki fark sorulur.', 'Önce ders kitaplarından bahsedelim.', 'Bir ders kitabının en iyi yanı, onu bir kere satın alıp üç ya da dört yıl boyunca kullanmaya devam edebilmenizdir.']


In [ ]:
%cd /content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos

/content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos


In [ ]:
import ffmpeg
from googletrans import Translator

def add_subtitle_to_video(soft_subtitle, subtitle_file, subtitle_language, input_video, input_video_name):
    video_input_stream = ffmpeg.input(input_video)
    subtitle_input_stream = ffmpeg.input(subtitle_file)
    output_video = f"output-{input_video_name}.mp4"
    subtitle_track_title = subtitle_file.replace(".srt", "")

    if soft_subtitle:
        stream = ffmpeg.output(
            video_input_stream, subtitle_input_stream, output_video, **{"c": "copy", "c:s": "mov_text"},
            **{"metadata:s:s:0": f"language={subtitle_language}",
               "metadata:s:s:0": f"title={subtitle_track_title}"}
        )
        ffmpeg.run(stream, overwrite_output=True)
    else:
        stream = ffmpeg.output(video_input_stream, output_video,
                               vf=f"subtitles={subtitle_file}")
        ffmpeg.run(stream, overwrite_output=True)

In [ ]:


# Initialize translator
translator = Translator()
target_language = 'tr'

# Read the subtitle file
with open('/content/drive/MyDrive/NLP_Project/testMarlin/MARLIN/test/videos/sub.srt', 'r', encoding='utf-8') as file:
    lines = file.readlines()


In [ ]:

translated_lines = []
text_block = ""
time_block = ""


In [ ]:

for line in lines:
    if '-->' in line:
        if text_block:
            translated = translator.translate(text_block.strip(), dest=target_language).text
            translated_lines.append(text_block.strip())
            translated_lines.append(translated + "\n")
        text_block = ""
        time_block = line.strip()
        translated_lines.append(time_block)
    elif line.strip().isdigit():
        translated_lines.append(line.strip())
    else:
        text_block += line


In [ ]:

if text_block:
    translated = translator.translate(text_block.strip(), dest=target_language).text
    translated_lines.append(text_block.strip())
    translated_lines.append(translated + "\n")

# Write translated subtitles to a new file
translated_subtitle_file = 'translated_subtitles.srt'
with open(translated_subtitle_file, 'w', encoding='utf-8') as file:
    for line in translated_lines:
        file.write(line + '\n')


In [ ]:
soft_subtitle = False  # Set to True for soft subtitles, False for hard subtitles

add_subtitle_to_video(soft_subtitle, translated_subtitle_file, subtitle_language, input_video, input_video_name)


In [ ]:
%cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
loaded_model = load_model('sentiment_classification_based_on_text.h5')

In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Example compilation


In [ ]:
# Define a dictionary of chat word mappings
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [ ]:
# Define a mapping dictionary
label_mapping = {'surprise':0, 'worry':1, 'hate':2, 'happiness':3, 'sadness':4, 'anger':5, 'neutral':6}

In [ ]:
# Open the file for reading
with open("/content/drive/MyDrive/NLP_Project/sentences_video03.txt", "r") as text_file:
  # Create an empty list to store the lines
  test = []
  # Read each line from the file
  for line in text_file:
    # Remove any trailing newline character
    line = line.strip()
    # Add the line to the new array
    test.append(line)

In [ ]:
test

['Hey, this is Ken from AES.',
 "Sometimes we're asked about the difference between what we do and a textbook.",
 "Let's talk about textbooks first.",
 'The best thing about a textbook is you can buy it once and you can keep using it for three or four years.']

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:


def preprocess_text(text):

    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in [word.lower() for word in list(chat_words.keys())]:
            words[i] = chat_words[word]
    text = ' '.join(words)

    # Remove non-alphabetic and non-space characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove stopwords
    stop = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stop])

    # Convert to lowercase
    text = text.lower()

    # Remove digits
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)

    # Remove non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Initialize the PorterStemmer
    stemmer = PorterStemmer()
    # Function to apply stemming to a text
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    text =  ' '.join(stemmed_words)


    return text



In [ ]:
test = [preprocess_text(text) for text in test]

In [ ]:
test

['hey ken ae',
 'sometim ask differ textbook',
 'let talk textbook first',
 'the best thing textbook buy keep use three four year']

In [ ]:
# Later, when you want to use the saved tokenizer:
# Load the tokenizer from the file
with open('/content/drive/MyDrive/NLP_Project/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Now you can use the loaded tokenizer to tokenize your text
tokenized_text = tokenizer.texts_to_sequences(test)

In [ ]:
tokenized_text

[[1482, 6015, 17178],
 [88, 127, 236, 4725],
 [116, 86, 4725, 110],
 [37898, 235, 14, 4725, 611, 100, 46, 527, 1008, 43]]

In [ ]:
test_sequence = tokenized_text

In [ ]:
test_padded = pad_sequences(test_sequence, maxlen=maxlen, padding='post')

In [ ]:
# Predictions On Test For Confustion Matrix
y_pred = loaded_model.predict(test_padded)
print(y_pred)

1/1 [==============================] - 1s 670ms/step
[[9.30251427e-13 1.30294115e-14 5.33976960e-11 1.08259313e-11
  4.80390424e-12 3.02956336e-13 1.00000000e+00]
 [1.76779850e-18 2.39250845e-20 2.18692492e-17 9.56213426e-18
  5.25495535e-16 3.04875914e-15 1.00000000e+00]
 [3.10017262e-16 9.65066451e-19 1.74370986e-15 9.63119761e-14
  6.79040916e-13 1.08939897e-12 1.00000000e+00]
 [2.95494966e-21 7.75759200e-22 2.70916767e-18 4.81240032e-18
  2.67075259e-16 9.00005197e-17 1.00000000e+00]]


In [ ]:
import torch
from torch.nn import functional as F  # Import functional for softmax


# Create a PyTorch tensor
tensor = torch.tensor(y_pred)

# Apply softmax along the specified dimension (default is dim=1)
softmax_output = F.softmax(tensor, dim=1)

print(softmax_output)

tensor([[0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118],
        [0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118],
        [0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118],
        [0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118]])


In [ ]:

# Convert softmax_output tensor to numpy array
softmax_array = softmax_output.cpu().detach().numpy()


In [ ]:
%cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:

# Save the softmax arrays to a text file
with open("softmax_output_of_text.txt", "w") as file:
    for array in softmax_array:
        file.write(' '.join([str(elem) for elem in array]) + '\n')

In [ ]:

y_pred = np.argmax(y_pred, axis=1)
y_pred

array([6, 6, 6, 6])

In [ ]:
def get_key_from_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # If the value is not found

In [ ]:
results = [get_key_from_value(label_mapping, label) for label in y_pred]
results

['Nneutral', 'Nneutral', 'Nneutral', 'Nneutral']

In [ ]:
%cd /content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER

/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER


In [ ]:
!pip install -r req.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch.utils.data as data
from torchvision import transforms
import torch
import os
import argparse
from data_preprocessing.dataset_raf import RafDataSet
from data_preprocessing.dataset_affectnet import Affectdataset
from data_preprocessing.dataset_affectnet_8class import Affectdataset_8class
import random

from utils import *
from models.emotion_hyp import pyramid_trans_expr
from sklearn.metrics import confusion_matrix
from data_preprocessing.plot_confusion_matrix import plot_confusion_matrix

import cv2


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = str('0,1')
print("Work on GPU: ", os.environ['CUDA_VISIBLE_DEVICES'])

Work on GPU:  0,1


In [ ]:

def save_random_frame_from_section(video_path, output_folder, section_start, section_end, section_number):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(section_start * fps)
    end_frame = int(section_end * fps)

    random_frame = random.randint(start_frame, end_frame)
    cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame)

    ret, frame = cap.read()
    if ret:
        frame_filename = os.path.join(output_folder, f"frame_section_{section_number}.jpg")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved frame from section {section_number} to {frame_filename}")
    else:
        print(f"Failed to capture frame from section {section_number}")

    cap.release()

In [ ]:
def extract_frames(video_path, output_folder, num_sections=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps

    section_duration = duration / 4

    sections_to_extract = random.sample(range(1, 5), min(num_sections, 4))

    for section_number in sections_to_extract:
        section_start = (section_number - 1) * section_duration
        section_end = section_number * section_duration
        save_random_frame_from_section(video_path, output_folder, section_start, section_end, section_number)

    cap.release()

In [ ]:

num_sections = sentences_num

In [ ]:
extract_frames(video_path, output_folder, num_sections)


Saved frame from section 1 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_1.jpg
Saved frame from section 2 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_2.jpg
Saved frame from section 3 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_3.jpg
Saved frame from section 4 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_4.jpg


In [ ]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            # Read the image
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Could not read image: {img_path}")
    return images

images_array = read_images_from_folder(output_folder)

# Now you have all the images stored in the `images_array` list

In [ ]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
model = pyramid_trans_expr(img_size=224, num_classes=num_classes, type="large")

load_weight 304


In [ ]:
def add_gaussian_noise(image_array, mean=0.0, var=30):
    std = var**0.5
    noisy_img = image_array + np.random.normal(mean, std, image_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img_clipped

def flip_image(image_array):
    return cv2.flip(image_array, 1)

In [ ]:
aug_func = [flip_image, add_gaussian_noise]

In [ ]:
index = 0
for img in images_array:
  i = random.randint(0, 1)
  img = aug_func[i](img)
  img = transform(img.copy())
  images_array[index] = img
  index += 1

In [ ]:
test_loader = torch.utils.data.DataLoader(images_array,
                                             batch_size=32,
                                             num_workers=2,
                                             shuffle=False,
                                             pin_memory=True)

In [ ]:
print("Loading pretrained weights...", "/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth")
checkpoint = torch.load("/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth")
checkpoint = checkpoint["model_state_dict"]
model = load_pretrained_weights(model, checkpoint)

Loading pretrained weights... /content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth
load_weight 1309


In [ ]:
model = model.cuda()

In [ ]:
with torch.no_grad():
    bingo_cnt = 0
    model.eval()
    for batch_i, (imgs) in enumerate(test_loader):
        outputs, features = model(imgs.cuda())
        _, predicts = torch.max(outputs, 1)
        print(predicts)
        print(outputs)

tensor([3, 2, 2, 0], device='cuda:0')
tensor([[-0.4358, -1.0675,  0.3217,  2.8434, -0.8518, -0.3195, -0.4349],
        [ 0.2487, -1.2416,  1.9898,  0.5242, -0.4681,  0.1476, -0.6603],
        [ 0.9859, -1.3570,  1.5785,  0.0784, -0.4941,  0.0539, -0.7567],
        [ 1.3812, -1.3101,  1.2315, -0.0459, -0.3129,  0.3867, -1.1606]],
       device='cuda:0')


In [ ]:
import torch
import torch.nn.functional as F


In [ ]:
# Apply softmax along the specified dimension (default is dim=1)
softmax_output = F.softmax(outputs, dim=1)
softmax_output

tensor([[0.0303, 0.0161, 0.0646, 0.8046, 0.0200, 0.0340, 0.0303],
        [0.0996, 0.0224, 0.5680, 0.1312, 0.0486, 0.0900, 0.0401],
        [0.2436, 0.0234, 0.4406, 0.0983, 0.0555, 0.0959, 0.0426],
        [0.3570, 0.0242, 0.3074, 0.0857, 0.0656, 0.1321, 0.0281]],
       device='cuda:0')

In [ ]:

# Convert softmax_output tensor to numpy array
softmax_array = softmax_output.cpu().detach().numpy()


In [ ]:
%cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:

# Save the softmax arrays to a text file
with open("softmax_output_of_video.txt", "w") as file:
    for array in softmax_array:
        file.write(' '.join([str(elem) for elem in array]) + '\n')

In [ ]:
import torch

def read_data(filename):
  """Reads data from a file into a PyTorch tensor."""
  with open(filename, 'r') as f:
    data_lines = f.readlines()  # Read all lines

  data = []
  for line in data_lines:
    data.append([float(x) for x in line.strip().split()])  # Split and convert each line

  return torch.tensor(data)  # Create a 2D tensor

def element_wise_dot_product(file1, file2):
  """Reads data from files, performs element-wise dot product, and applies softmax."""
  data1 = read_data(file1)
  data2 = read_data(file2)
  print(data1)
  print(data2)
  # Ensure data has the same dimensions
  if data1.size() != data2.size():
    raise ValueError("Data in files must have the same dimensions for element-wise multiplication.")

  # Calculate element-wise dot product
  element_wise_product = data1 * data2  # Use element-wise multiplication operator
  print(element_wise_product)
  # Apply softmax on each row (assuming data points are in rows)
  softmax = torch.nn.functional.softmax(element_wise_product, dim=1)

  return softmax


# Example usage (replace 'file1.txt' and 'file2.txt' with your actual file paths)
result = element_wise_dot_product(softmax_of_text, softmax_of_video)
print(result)


tensor([[0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118],
        [0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118],
        [0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118],
        [0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.1147, 0.3118]])
tensor([[0.0303, 0.0161, 0.0646, 0.8046, 0.0200, 0.0340, 0.0303],
        [0.0996, 0.0224, 0.5680, 0.1312, 0.0486, 0.0900, 0.0401],
        [0.2436, 0.0234, 0.4406, 0.0983, 0.0555, 0.0959, 0.0426],
        [0.3570, 0.0242, 0.3074, 0.0857, 0.0656, 0.1321, 0.0281]])
tensor([[0.0035, 0.0018, 0.0074, 0.0923, 0.0023, 0.0039, 0.0095],
        [0.0114, 0.0026, 0.0652, 0.0150, 0.0056, 0.0103, 0.0125],
        [0.0279, 0.0027, 0.0505, 0.0113, 0.0064, 0.0110, 0.0133],
        [0.0409, 0.0028, 0.0353, 0.0098, 0.0075, 0.0151, 0.0088]])
tensor([[0.1408, 0.1406, 0.1414, 0.1539, 0.1407, 0.1409, 0.1417],
        [0.1420, 0.1407, 0.1498, 0.1425, 0.1411, 0.1418, 0.1421],
        [0.1443, 0.1407, 0.1476, 0.1419, 0.1412, 0.1419, 0.1422],
       

In [ ]:
result_np = result.numpy()

In [ ]:
%cd /content/drive/MyDrive/NLP_Project
# Save the softmax arrays to a text file
with open("softmax_result.txt", "w") as file:
    for array in result_np:
        file.write(' '.join([str(elem) for elem in array]) + '\n')

/content/drive/MyDrive/NLP_Project


In [ ]:
import numpy as np
final_result = np.argmax(result, axis=1)
final_result

tensor([3, 2, 2, 0])

In [ ]:
# Define a mapping dictionary
label_mapping = {'surprise':0, 'worry':1, 'hate':2, 'happiness':3, 'sadness':4, 'anger':5, 'neutral':6}

In [ ]:
def get_key_from_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # If the value is not found

In [ ]:
res = [get_key_from_value(label_mapping, label) for label in final_result]
res

['happiness', 'hate', 'hate', 'surprise']

In [ ]:
%cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:
!pip install opencv-python-headless moviepy


In [ ]:
import cv2
from moviepy.editor import VideoFileClip, AudioFileClip

# Define your list of texts
text_list = res
input_video = input_video_to_add_sentiment  # Get the uploaded video filename
output_video_with_audio = "output_video_with_audio.mp4"

# Open the video file with OpenCV
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

# Calculate the duration each text should be displayed
num_texts = len(text_list)
display_duration = duration / num_texts
frames_per_text = int(display_duration * fps)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a temporary VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
temp_output = "temp_output.mp4"
out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))

current_text_index = 0
frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Calculate which text to display based on frame index
    current_text_index = frame_index // frames_per_text
    if current_text_index >= num_texts:
        current_text_index = num_texts - 1

    text = "Sentiment: " + text_list[current_text_index]

    # Put the text on the frame
    cv2.putText(frame, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

    frame_index += 1

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

# Use moviepy to add the original audio back to the processed video
original_video = VideoFileClip(input_video)
processed_video = VideoFileClip(temp_output)

final_video = processed_video.set_audio(original_video.audio)
final_video.write_videofile(output_video_with_audio, codec="libx264")




Moviepy - Building video output_video_with_audio.mp4.
MoviePy - Writing audio in output_video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready output_video_with_audio.mp4
